In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyserial


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import pytesseract
import mysql.connector
import time
import tkinter as tk
from tkinter import messagebox
import serial  # Para la comunicación con Arduino

# Configurar Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

#arduino = serial.Serial(port="COM3", baudrate=9600, timeout=1)  # Cambia "COM3" según corresponda

# Conexión a la base de datos MySQL
def setup_mysql_connection():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="pf_v4"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Función para enviar comandos al Arduino
def enviar_comando_arduino(comando):
    try:
        arduino.write(comando.encode())  # Enviar el comando como bytes
        time.sleep(0.1)  # Esperar un poco para asegurar la transmisión
    except Exception as e:
        print(f"Error al enviar comando al Arduino: {e}")

# Función para formatear la matrícula
def format_plate(plate_text):
    if len(plate_text) == 7 and '-' not in plate_text:
        plate_text = plate_text[:3] + '-' + plate_text[3:]
    return plate_text

# Preprocesar imagen para OCR
def preprocess_for_ocr(plate_img):
    gray_plate = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    blurred_plate = cv2.GaussianBlur(gray_plate, (5, 5), 0)
    _, thresh_plate = cv2.threshold(blurred_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh_plate

# Mostrar resultados en una ventana
from PIL import Image, ImageTk
import tkinter as tk

def mostrar_resultados_en_ventana(placa, status, persona=None, fecha_hora=None):
    ventana = tk.Tk()
    ventana.title("Resultado de Deteccion")
    ventana.geometry("500x500")

    # Ruta al logo
    logo_path = "C://Users//TUF GAMING F15//Downloads//logo tec.png"

    try:
        # Cargar y mostrar el logo
        logo_image = Image.open(logo_path)
        logo_image = logo_image.resize((200, 100), Image.LANCZOS)  # Redimensionar sin usar ANTIALIAS
        logo_photo = ImageTk.PhotoImage(logo_image)
        label_logo = tk.Label(ventana, image=logo_photo)
        label_logo.image = logo_photo
        label_logo.pack(pady=10)
    except Exception as e:
        print(f"Error al cargar el logo: {e}")

    # Información debajo del logo
    label_placa = tk.Label(ventana, text=f"Placa Detectada: {placa}", font=("Arial", 16))
    label_placa.pack(pady=15)

    label_status = tk.Label(ventana, text=f"Estado: {status}", font=("Arial", 16), fg="green" if status == "Autorizado" else "red")
    label_status.pack(pady=10)

    if status == "Autorizado" and persona and fecha_hora:
        label_persona = tk.Label(ventana, text=f"Propietario: {persona}", font=("Arial", 14))
        label_persona.pack(pady=5)

        label_fecha_hora = tk.Label(ventana, text=f"Fecha y Hora: {fecha_hora}", font=("Arial", 14))
        label_fecha_hora.pack(pady=5)

    boton_cerrar = tk.Button(ventana, text="Cerrar", command=ventana.destroy, font=("Arial", 14))
    boton_cerrar.pack(pady=30)

    ventana.mainloop()



# Detección y comparación de matrículas
def detect_license_plate_from_camera():
    conn = setup_mysql_connection()
    if conn is None:
        print("No se pudo conectar a la base de datos.")
        return

    cursor = conn.cursor()
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("No se puede acceder a la cámara.")
        return

    plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_russian_plate_number.xml')
    if plate_cascade.empty():
        print("No se pudo cargar el modelo de detección de placas.")
        return

    cv2.namedWindow("Deteccion de Placas", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Deteccion de Placas", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    print("Presiona 'q' para salir.")

    last_detection_time = 0  # Inicializar el tiempo de la última detección
    detection_interval = 5   # Intervalo en segundos entre lecturas

    while True:
        ret, frame = cap.read()
        if not ret:
            print("No se pudo recibir el cuadro de la cámara.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        plates = plate_cascade.detectMultiScale(gray, 1.1, 10)
        

        # Verificar si ha pasado suficiente tiempo desde la última detección
        current_time = time.time()
        if current_time - last_detection_time >= detection_interval:
            for (x, y, w, h) in plates:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                plate_img = frame[y:y+h, x:x+w]
                plate_img = preprocess_for_ocr(plate_img)

                try:
                    # Detectar el texto de la matrícula
                    plate_text = pytesseract.image_to_string(plate_img, config='--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789').strip()
                    if plate_text:
                        # Limpiar texto y agregar guion si es necesario
                        plate_text = ''.join(filter(str.isalnum, plate_text))  # Limpiar caracteres no válidos
                        plate_text = format_plate(plate_text)  # Formatear texto detectado

                        # Verificar si la placa ya existe en la tabla placadetectada
                        insert_query_placadetectada = '''
                            INSERT INTO placadetectada (placa_detectada)
                            SELECT %s
                            WHERE NOT EXISTS (
                                SELECT 1 FROM placadetectada WHERE placa_detectada = %s
                            )
                        '''
                        cursor.execute(insert_query_placadetectada, (plate_text, plate_text))
                        conn.commit()

                        # Consultar si la placa detectada coincide con algún vehículo
                        select_query = '''
                            SELECT v.id_vehiculo, d.nombre_docente AS propietario, NOW() AS fecha_hora
                            FROM vehiculo v
                            JOIN docente d ON v.id_docente = d.id_docente
                            WHERE v.placa = %s
                        '''
                        cursor.execute(select_query, (plate_text,))
                        result = cursor.fetchone()

                        if result:
                            id_vehiculo, propietario, fecha_hora = result
                        else:
                            id_vehiculo = propietario = fecha_hora = None

                        # Insertar en la tabla deteccion (Autorizado o No Autorizado)
                        insert_query_deteccion = '''
                            INSERT INTO deteccion (id_vehiculo, id_placa_detectada, fecha_hora)
                            VALUES (%s, (SELECT id_placa_detectada FROM placadetectada WHERE placa_detectada = %s), NOW())
                        '''
                        cursor.execute(insert_query_deteccion, (id_vehiculo, plate_text))
                        conn.commit()

                        # Mostrar resultado autorizado o no autorizado
                        status_text = "Autorizado" if id_vehiculo else "No autorizado"
                        color = (0, 255, 0) if id_vehiculo else (0, 0, 255)
                        cv2.putText(frame, f"{status_text}: {plate_text}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                        print(f"Matricula detectada: {plate_text} - {status_text}")

                        # Enviar comando al Arduino dependiendo del estado
                        comando_arduino = 'A' if id_vehiculo else 'D'
                        enviar_comando_arduino(comando_arduino)

                        # Mostrar resultados en la ventana
                        mostrar_resultados_en_ventana(plate_text, status_text, propietario, fecha_hora)

                        # Actualizar el tiempo de la última detección
                        last_detection_time = current_time
                        break  # Salir del bucle después de procesar una matrícula
                except Exception as e:
                    print(f"Error al procesar la imagen de la placa: {e}")

        cv2.imshow('Deteccion de Placas', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cursor.close()
    conn.close()
# Llamar a la función
detect_license_plate_from_camera()

Presiona 'q' para salir.


Matricula detectada: 2424 - No autorizado
Error al enviar comando al Arduino: name 'arduino' is not defined


Matricula detectada: B3424I - No autorizado
Error al enviar comando al Arduino: name 'arduino' is not defined


Matricula detectada: ABB-3424 - Autorizado
Error al enviar comando al Arduino: name 'arduino' is not defined
